In [ ]:
! pip install google-genai

### Setting API Value

In [ ]:
API_KEY = "<ADD your GEMINI API KEY>"
MODEL_NAME = "gemini-2.5-flash"


## CODE SESSION: Building ReAct Agent

In [ ]:
# BLOCK 1: Just talking to an LLM
# This is where we START - no agent, no planning, just a simple call

import google.generativeai as genai

genai.configure(api_key=API_KEY)
model = genai.GenerativeModel(MODEL_NAME)

model

genai.GenerativeModel(
    model_name='models/gemini-2.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [ ]:
# Simple question
response = model.generate_content("What's 25 + 37?")
print(response.text)

25 + 37 = 62


In [ ]:
# Problem: What if we need the LLM to use a calculator?
# It might hallucinate the answer!
response = model.generate_content("What's 8928474761 × 2938472232425?")
print(response.text)

To calculate 8928474761 × 2938472232425:

The product is **26,233,036,499,313,467,666,225**


In [ ]:
# BLOCK 2: Let's give LLM a calculator tool
# But we control when to use it

def calculator(expression):
    """A simple calculator tool"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except:
        return "Error: Invalid expression"


In [ ]:

# Now let's use it
# task = "What is capital of bangladesh??"
task = "What's 8928474761 × 2938472232425?"

# Step 1: Ask LLM what to do
response = model.generate_content(f"""
Task: {task}

You have access to a calculator tool.
Should you use it? If yes, what expression should you calculate?

Reply in this format:
USE_TOOL: <expression>
or
NO_TOOL: <direct answer>
""")

print("LLM says:", response.text)

LLM says: USE_TOOL: 8928474761 * 2938472232425


In [ ]:

import re

def extract_expression_for_calculator(txt):
    """
    Extract mathematical expressions from text.
    Looks for patterns like: number operator number
    where numbers can contain commas (e.g., 29,123)
    and operators are: +, -, *, /, **, //, %

    Args:
        txt: String that may contain a mathematical expression

    Returns:
        The mathematical expression, or None if not found
    """
    # First try to extract from calculator(...) pattern
    calc_pattern = r'calculator\s*\(([^)]+)\)'
    calc_match = re.search(calc_pattern, txt)
    if calc_match:
        return calc_match.group(1).strip()

    # If no calculator pattern, look for mathematical expression pattern
    # Pattern explanation:
    # ([\d,]+) - captures numbers with optional commas
    # \s* - optional whitespace
    # ([+\-*/%]|\*\*|//) - captures operators: +, -, *, /, **, //, %
    # \s* - optional whitespace
    # ([\d,]+) - captures second number with optional commas
    expr_pattern = r'([\d,]+)\s*([+\-*/%]|\*\*|//)\s*([\d,]+)'

    match = re.search(expr_pattern, txt)
    if match:
        # Reconstruct the expression with original spacing preserved
        num1, operator, num2 = match.groups()
        return match.group(0)

    return None

extract_expression_for_calculator(txt="ACTION: calculator(8928474761 * 2938472232425)")

'8928474761 * 2938472232425'

In [ ]:
## USE_TOOL: 8928474761 * 2938472232425

# Step 2: Parse and execute
if "USE_TOOL:" in response.text:
    expression = extract_expression_for_calculator(response.text) # response.text.split("USE_TOOL:")[1].strip()
    result = calculator(expression)
    print("Tool result:", result)

# PAUSE: "This works, but it's manual. How do we automate this?"

Tool result: Result: 26236075163105938325425


In [ ]:
# BLOCK 3: Add REASONING before acting
# This is the "Re" in ReAct (Reasoning + Acting)

def think_then_act(task):
    """Agent thinks about what to do first"""

    # Step 1: THINK - What's the plan?
    think_prompt = f"""
Task: {task}

Think step by step:
1. What do I need to do?
2. Do I need any tools?
3. What's my next action?

Don't must not try the calculation by yourself, if it is a tough one. Just say what your next step should be.
"""

    reasoning = model.generate_content(think_prompt)
    print("💭 THINKING:")
    print(reasoning.text)
    print("\n" + "="*50 + "\n")

    # Step 2: ACT - Decide on action
    action_prompt = f"""
Based on this reasoning:
{reasoning.text}

What action should I take?

Reply with:
ACTION: calculator(<expression>)
or
ACTION: answer(<direct answer>)
"""
    print("Action prompt")
    # print("%%"*15)
    # print(action_prompt)
    # print("%%"*15)
    action = model.generate_content(action_prompt)
    print("🎬 ACTION:")
    print(action.text)


    if action.text.startswith("ACTION: calculator"):
        expression = extract_expression_for_calculator(action.text)
        print("Calculator expression:", expression)
        result = calculator(expression)
        print("Tool result:", result)

    return reasoning.text, action.text

# Test it
reasoning_text, action_text = think_then_act("What is 8928474761 × 2938472232425?")

# print("\n\n" + "*"*30)
# print("REASONING TEXT:")
# print(reasoning_text)

# print("\n" + "*"*30)
# print("ACTION TEXT:")
# print(action_text)

# PAUSE: "Good! But what if we need multiple steps?"

💭 THINKING:
Here's my thought process:

1.  **What do I need to do?**
    I need to calculate the product of 8928474761 and 2938472232425.

2.  **Do I need any tools?**
    Yes, these are very large numbers (10 digits and 13 digits respectively). Performing this multiplication manually would be exceedingly complex and time-consuming, with a high probability of error. A standard handheld calculator likely wouldn't have the capacity for both the input numbers and the resulting product. I would need a computational tool designed for large number arithmetic, such as a programming language (e.g., Python), a scientific calculator with large number support, or a web-based computational engine (e.g., Google calculator, Wolfram Alpha).

3.  **What's my next action?**
    My next action would be to use a suitable computational tool to perform the multiplication. I would input "8928474761 * 2938472232425" into such a tool and retrieve the result.


Action prompt
🎬 ACTION:
ACTION: calculator(89284

In [ ]:

reasoning_text, action_text = think_then_act("What is the national fruit of Bangladesh?")

# print("\n\n" + "*"*30)
# print("REASONING TEXT:")
# print(reasoning_text)

# print("\n" + "*"*30)
# print("ACTION TEXT:")
# print(action_text)

💭 THINKING:
1.  **What do I need to do?** Identify the national fruit of Bangladesh.
2.  **Do I need any tools?** Yes, I need access to information (my knowledge base or a search engine).
3.  **What's my next action?** Retrieve the information regarding the national fruit of Bangladesh.

The national fruit of Bangladesh is the **Jackfruit**.


Action prompt
🎬 ACTION:
ACTION: answer(Jackfruit)


In [ ]:
# BLOCK 4: Multiple reasoning-action cycles
# A true ReAct loop!

def simple_react_agent(task, max_steps=3):
    """
    A basic ReAct agent that can take multiple steps
    """

    tools = {
        "calculator": lambda expr: eval(expr),
        "search": lambda query: f"[Simulated search results for: {query}]"
    }

    history = []

    for step in range(max_steps):
        print(f"\n{'='*60}")
        print(f"STEP {step + 1}")
        print('='*60)

        # REASONING: What should I do next?
        reasoning_prompt = f"""
Task: {task}

What I've done so far:
{chr(10).join([f"- {h}" for h in history]) if history else "Nothing yet"}

Available tools:
- calculator(expression): for math calculations
- search(query): to search for information

Think: What should I do next? Am I done?[if not done, don't use the word `done` or `complete` in the answer.]

Your reasoning:"""

        reasoning = model.generate_content(reasoning_prompt)
        thought = reasoning.text
        print(f"💭 THOUGHT:\n{thought}\n")

        # Check if done
        if "done" in thought.lower() or "complete" in thought.lower():
            print("✅ Task complete!")
            break

        # ACTION: What action to take?
        action_prompt = f"""
Based on your reasoning:
{thought}

Choose ONE action:
- calculator(expression)
- search(query)
- answer(final_answer)

Reply ONLY with the action in this exact format:
TOOL: calculator
INPUT: 2+2

or
TOOL: answer
INPUT: The answer is X
"""

        action_response = model.generate_content(action_prompt)
        print(f"🎬 ACTION:\n{action_response.text}\n")

        # EXECUTE the action
        try:
            if "TOOL:" in action_response.text and "INPUT:" in action_response.text:
                lines = action_response.text.strip().split('\n')
                tool_name = lines[0].split("TOOL:")[1].strip()
                tool_input = lines[1].split("INPUT:")[1].strip()

                if tool_name == "answer":
                    print(f"📊 FINAL ANSWER: {tool_input}")
                    break
                elif tool_name in tools:
                    result = tools[tool_name](tool_input)
                    print(f"📊 OBSERVATION: {result}")
                    history.append(f"Used {tool_name}({tool_input}) → {result}")
        except Exception as e:
            print(f"⚠️ Error executing action: {e}")
            history.append(f"Error: {e}")

    return history

# Test with a simple task
simple_react_agent("What is 150 + 275 + 89?")



STEP 1
💭 THOUGHT:
I need to calculate the sum of 150, 275, and 89. I can use the `calculator` tool for this.

**Tool Call:**
calculator("150 + 275 + 89")

🎬 ACTION:
TOOL: calculator
INPUT: 150 + 275 + 89

📊 OBSERVATION: 514

STEP 2
💭 THOUGHT:
The calculation has been performed and the correct sum is found. No further steps are needed for this task.

🎬 ACTION:
TOOL: answer
INPUT: The calculation has been performed and the correct sum is found. No further steps are needed for this task.

📊 FINAL ANSWER: The calculation has been performed and the correct sum is found. No further steps are needed for this task.


['Used calculator(150 + 275 + 89) → 514']

In [ ]:

# Test with a complex task
simple_react_agent("Search for AI agents, then calculate how many days until 2025")


STEP 1
💭 THOUGHT:
I need to perform two main tasks: search for AI agents and calculate the days until 2025.

First, I will use the `search` tool to find information about AI agents.
Second, to calculate the days until 2025, I will assume the question means the number of days in the year 2024, as 2024 is the year immediately preceding 2025. I need to determine if 2024 is a leap year for this calculation. 2024 is divisible by 4, so it is a leap year, meaning it has 366 days. I will use the `calculator` tool to confirm this or perform any necessary date arithmetic if a specific start date were provided (which it isn't).

Action:
1. Search for "AI agents".
2. Calculate the number of days in 2024 (which is a leap year, so 366 days).

```json
{"tool": "search", "parameters": {"query": "AI agents"}}
```

🎬 ACTION:
TOOL: calculator
INPUT: 365 + 1

📊 OBSERVATION: 366

STEP 2
💭 THOUGHT:
Your reasoning:
The user has two tasks: 1) Search for AI agents, and 2) Calculate how many days until 2025.
S

['Used calculator(365 + 1) → 366',
 'Used search(AI agents) → [Simulated search results for: AI agents]']

In [ ]:
# BLOCK 5: Production-ready ReAct Agent
# With better error handling and structure

import json
from datetime import datetime

class ReActAgent:
    """
    A complete ReAct agent implementation
    """

    def __init__(self, model_name=MODEL_NAME):
        self.model = genai.GenerativeModel(
            model_name,
            safety_settings={
                'HARASSMENT': 'block_none',
                'HATE_SPEECH': 'block_none',
                'SEXUALLY_EXPLICIT': 'block_none',
                # 'DANGEROUS_CONTENT': 'block_none',
            })
        self.tools = self._initialize_tools()
        self.history = []

    def _initialize_tools(self):
        """Define available tools"""
        return {
            "calculator": {
                "function": lambda expr: str(eval(expr)),
                "description": "Performs mathematical calculations"
            },
            "search": {
                "function": lambda query: f"Search results for '{query}': [AI agents are software programs that use LLMs...]",
                "description": "Searches the web for information"
            },
            "get_date": {
                "function": lambda: datetime.now().strftime("%Y-%m-%d"),
                "description": "Gets the current date"
            }
        }

    def _reason(self, task):
        """Reasoning step - figure out what to do"""

        tools_desc = "\n".join([
            f"- {name}: {info['description']}"
            for name, info in self.tools.items()
        ])

        history_text = "\n".join([
            f"Step {i+1}: {step['summary']}"
            for i, step in enumerate(self.history)
        ]) if self.history else "No steps taken yet"

        prompt = f"""
You are a helpful AI agent working on this task:
TASK: {task}

HISTORY:
{history_text}

AVAILABLE TOOLS:
{tools_desc}

THINK CAREFULLY:
1. What have I accomplished?
2. What's the next logical step?
3. Which tool (if any) should I use?
4. Am I done with the task?

Provide your reasoning:"""

        response = self.model.generate_content(prompt)
        return response.text

    def _decide_action(self, reasoning, task):
        """Decide what action to take based on reasoning"""

        prompt = f"""
Task: {task}
Your reasoning: {reasoning}

Based on your reasoning, decide your next action.

Respond in this EXACT JSON format:
{{
    "action_type": "use_tool" or "final_answer",
    "tool_name": "calculator" or "search" or "get_date" or null,
    "tool_input": "the input for the tool" or null,
    "final_answer": "your final answer if done" or null
}}

JSON:"""

        response = self.model.generate_content(prompt)

        # Parse JSON
        try:
            # Extract JSON from response
            text = response.text
            if "```json" in text:
                text = text.split("```json")[1].split("```")[0]
            elif "```" in text:
                text = text.split("```")[1].split("```")[0]

            return json.loads(text.strip())
        except:
            # Fallback if JSON parsing fails
            return {
                "action_type": "final_answer",
                "final_answer": response.text
            }
    def _execute_tool(self, tool_name, tool_input):
        """Execute a tool and return the result"""

        if tool_name not in self.tools:
            return f"Error: Tool '{tool_name}' not found"

        try:
            tool_function = self.tools[tool_name]["function"]

            # Handle tools that don't need input
            if tool_input is None or tool_input == "":
                result = tool_function()
            else:
                result = tool_function(tool_input)

            return result
        except Exception as e:
            return f"Error executing {tool_name}: {str(e)}"

    def run(self, task, max_steps=5, verbose=True):
        """
        Run the ReAct agent on a task
        """

        if verbose:
            print(f"\n{'='*70}")
            print(f"🎯 TASK: {task}")
            print('='*70)

        for step in range(max_steps):
            if verbose:
                print(f"\n--- STEP {step + 1} ---")

            # 1. REASON
            reasoning = self._reason(task)
            if verbose:
                print(f"\n💭 REASONING:\n{reasoning}")

            # 2. DECIDE ACTION
            action = self._decide_action(reasoning, task)
            if verbose:
                print(f"\n🎬 ACTION: {json.dumps(action, indent=2)}")

            # 3. EXECUTE
            if action["action_type"] == "final_answer":
                if verbose:
                    print(f"\n✅ FINAL ANSWER:\n{action['final_answer']}")
                return action["final_answer"]

            elif action["action_type"] == "use_tool":
                tool_name = action["tool_name"]
                tool_input = action["tool_input"]

                result = self._execute_tool(tool_name, tool_input)

                if verbose:
                    print(f"\n📊 OBSERVATION:\n{result}")

                # Add to history
                self.history.append({
                    "reasoning": reasoning,
                    "action": action,
                    "result": result,
                    "summary": f"Used {tool_name} with input '{tool_input}' → {result}"
                })

        return "Max steps reached without final answer"

# TEST THE COMPLETE AGENT!

# Complex test
print("\n\n" + "="*70)
print("SIMPLE TASK TEST")
print("="*70)
# Simple test
agent = ReActAgent()
agent.run("What is 456 + 789?")




SIMPLE TASK TEST

🎯 TASK: What is 456 + 789?

--- STEP 1 ---

💭 REASONING:
I need to calculate the sum of 456 and 789. The `calculator` tool is suitable for this.

**Tool Call:**

```json
{
  "tool_code": "print(calculator.calculate(456 + 789))"
}
```

🎬 ACTION: {
  "action_type": "use_tool",
  "tool_name": "calculator",
  "tool_input": "456 + 789",
  "final_answer": null
}

📊 OBSERVATION:
1245

--- STEP 2 ---

💭 REASONING:
I have already calculated the sum of 456 + 789 as 1245 using the calculator.
I am done with the task.

The final answer is $\boxed{1245}$

🎬 ACTION: {
  "action_type": "final_answer",
  "tool_name": null,
  "tool_input": null,
  "final_answer": "1245"
}

✅ FINAL ANSWER:
1245


'1245'

In [ ]:
# Complex test
print("\n\n" + "="*70)
print("COMPLEX TASK TEST")
print("="*70)

agent2 = ReActAgent()
agent2.run("What is today's date, and how many days until New Year 2026?")



COMPLEX TASK TEST

🎯 TASK: What is today's date, and how many days until New Year 2026?

--- STEP 1 ---

💭 REASONING:
I need to find out today's date first, then calculate the number of days until January 1, 2026.

1. **Get today's date:** Use the `get_date` tool.
2. **Calculate days remaining:** Once I have today's date, I will need to determine the number of days between that date and January 1, 2026. This will involve counting days in the remaining months of the current year, all days in 2025, and then adding them up. The `calculator` tool will be useful for summing these days.

**Tool:** `get_date`

🎬 ACTION: {
  "action_type": "use_tool",
  "tool_name": "get_date",
  "tool_input": null,
  "final_answer": null
}

📊 OBSERVATION:
2025-10-14

--- STEP 2 ---

💭 REASONING:
*   **What have I accomplished?**
    I have successfully retrieved today's date, which is 2025-10-14.

*   **What's the next logical step?**
    The next step is to calculate the number of days between 2025-10-14 a

'2025-10-14, and there are 78 days until New Year 2026.'